<img src="icons/icon.png" width="50"/> ![Badge](https://img.shields.io/badge/Workshop%20SMLMS%202025-FF1010?style=for-the-badge)
## Single Molecule Tracking and Diffusion

 *[Daniel Sage](mailto:daniel.sage@epfl.ch?subject=Single%20Molecule%20Tracking%20and%20Diffusion), Ecole Polytechnique Fédérale de Lausanne, Switerzland*


In [ ]:
try: 
  import deeptrack #Install deeptrack if not yet installed
except ImportError: 
  !pip install deeptrack

try: 
  import andi_datasets #Install andi_datasets if not yet installed
except ImportError: 
  !pip install andi_datasets

import numpy as np
import random
import imageio
import matplotlib.pyplot as plt
import deeptrack as dt
from andi_datasets.models_phenom import models_phenom
from andi_datasets.utils_videos import transform_to_video, play_video

# Simulation of anomalous models 

The notebook is a simplified version of the code of provided by *Gorka Muñoz-Gil* on the ANDI2 Challenge GitHub.
`https://github.com/AnDiChallenge/andi_datasets 

Generating N trajectories from one of the five models defined in ANDI.



### Generating trajectories

In [ ]:
T = 100       # Number of time steps (frames)
N = 20        # Number of particles (trajectories)
L = 1.5 * 128 # Length of the box (pixels) -> extending fov by 1.5 times
D = 0.1       # Diffusion coefficient (pixels^2/frame)
model = 1     # ID of the model in the below list

models_dict = {'single_state':0, 'multi_state':1, 'dimerization':2, 'confinement':3, 'immobile_traps':4}

if model == models_dict['single_state']:
    trajs, labels = models_phenom().single_state(N=N, L=L, T=T, alphas=1, Ds=1)
    
elif model == models_dict['multi_state']:
    trajs, labels = models_phenom().multi_state(N=N, L=L, T=T, alphas=[[1, 0], [1, 0]], Ds=[[1, 0], [0.1, 0]],  
                    M=[[0.9 , 0.1],[0.1 ,0.9]]) # Transition matrix between diffusive states

elif model == models_dict['dimerization']:
    trajs, labels = models_phenom().dimerization(N=N, L=L, T=T, alphas=[0.7, 0.1], Ds=[10 * D, 0],
                    Pb=1, Pu=0) # Binding and unbinding probability
    
elif model == models_dict['confinement']:
    trajs, labels = models_phenom().confinement(N=N, L=L, T=T, alphas=[[1, 0], [1, 0]], Ds=[[1, 0], [0.1, 0]], 
                    Nc = 10) # Number of compartments

elif model == models_dict['immobile_traps']:
    trajs, labels = models_phenom().immobile_traps(N=N, L=L, T=T, alphas=[1, 0], Ds=[1, 0],
                    Pu = 0.1, Pb = 0.01, Nt = 10) # Number of traps

# trajs dimension to T,N,2

plt.figure(figsize=(5, 5))
for i in range(N):
    plt.plot(trajs[:,i,1], trajs[:,i,0])
plt.xlim(0, 256)
plt.ylim(0, 256)
plt.axis("square")
ticks = np.arange(0, 257, 64)
plt.xticks(ticks)
plt.yticks(ticks)
plt.gca().invert_yaxis()
plt.show()

### Generating frames

In [ ]:
video = transform_to_video(
    np.moveaxis(trajs, 0, 1), 
    particle_props={"particle_intensity": [1000, 0]},
    background_props={"background_mean": 100},
    optics_props={"output_region" : [0, 0, 256, 256]   } # [x, y, x + width, y + height]
)
play_video(video)